In [5]:
import pandas as pd
import sqlite3
import polygon
import fredapi as Fred
import requests

##########################
import yfinance

# Options Client

In [ ]:
class OptionsClient:
    """
    dependencies = [pandas, polygon]
    """
    def __init__(self, api_key):
        self.client = polygon.RESTClient(api_key=api_key)

    def get_list_from_ticker(self, ticker, raw=False):
        contracts = []
        for i in self.client.list_options_contracts(underlying_ticker=ticker.upper(), limit=1000):
            contracts.append(i)
        
        if raw:
            df_raw = pd.DataFrame(contracts)
            return df_raw
        else:
            df = pd.DataFrame(contracts)
            df.drop(columns=[
                'additional_underlyings',
                'cfi',
                'correction',
                'primary_exchange',
                'shares_per_contract',
                'exercise_style',
                'underlying_ticker'
                ], axis=1, inplace=True)
            df.rename({
                'contract_type':'type',
                'ticker':'contract_ticker',
                'underlying_ticker':'stock_ticker'
                }, axis=1, inplace=True)
            df['expiration_date'] = pd.to_datetime(df['expiration_date'])
            df['ticker'] = ticker.upper()
            return df
        
    def get_aggs(self, contract_ticker, raw=False):
        lists = []
        for c in self.client.list_aggs(contract_ticker, 1, 'day', '2000-01-01', '2025-01-01', limit=5000):
            lists.append(c)
        df = pd.DataFrame(lists)
        if df.empty:
            return "no data"
        else:
            df['date'] = df['timestamp'].astype('datetime64[ms]')
            df['date'] = pd.to_datetime(df['date'].dt.date)
            df.drop(['timestamp', 'transactions', 'otc'], axis=1, inplace=True)
            df['contract_ticker'] = contract_ticker
            return df

# DB interactions

In [ ]:
class DataBaseClient:
    """
    dependencies = [pandas, sqlite3]
    """
    def __init__(self, db_name):
        self.db = db_name
        self.conn = sqlite3.connect(db_name)

    def data_query(self, sql_string):
        df = pd.read_sql(sql_string, self.conn)
        if 'date' in df.columns:
            df['date'] = pd.to_datetime(df['date'])
            return df
        elif 'expiration_date' in df.columns:
            df['expiration_date'] = pd.to_datetime(df['expiration_date'])
            return df
        else:
            return df
        
    def data_add(self, df, table_name):
        df.to_sql(table_name, self.conn, if_exists='append', index=False)

    def get_table_names(self):
        df = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", self.conn)
        return df 
    
    def close_db(self):
        self.conn.close()

    def delete_table(self, table_name):
        self.conn.execute(f"DROP TABLE {table_name}")

# Fred Client

In [ ]:
class FredClient:
    def __init__(self, api_key):
        self.fred = Fred(api_key=api_key)

    def fred_data_clean_(self, data):
        df = pd.DataFrame(data, columns=['value'])
        df.reset_index(inplace=True)
        df.rename(columns={'index': 'date'}, inplace=True)
        df['date'] = pd.to_datetime(df['date'])
        return df

    def get_series(self, series, raw=False):
        data = self.fred.get_series(series)
        if raw:
            return data
        df = self.fred_data_clean_(data)
        return df
    
    def get_fed_funds_rate(self, raw=False):
        if raw is True:
            df_full = self.get_series('DFF', raw=raw)
            return df
        else:
            df = self.get_series('DFF', raw=raw)
            df = df[df['date'].dt.year >= 2000]
            df.reset_index(drop=True, inplace=True)
            return df

# Yahoo Finance

In [ ]:
class YahooIndexClient:
    def __init__(self):
        pass

    def get_index(self, ticker, raw=False):
        ticker_object = yfinance.Ticker(ticker.upper())
        df = ticker_object.history(period='max')
        df.reset_index(inplace=True)
        df.drop(columns=['Dividends', 'Stock Splits', 'Volume'], axis=1, inplace=True)
        df['Date'] = pd.to_datetime(df['Date'])
        df['Date'] = pd.to_datetime(df['Date'].dt.date)
        df.rename(columns={
            'Date':'date',
            'Open':'open',
            'High':'high',
            'Low':'low',
            'Close':'close'
            }, inplace=True)
        df = df[df['date'].dt.year >= 2000]
        return df


# Old Finpy

In [ ]:

def binary_up_down_(ovnChange):
    if ovnChange >= 0:
        return 1
    else:
        return 0

def data_shifts_(df):
    df['futureClose'] = df['close'].shift(1)
    df['ovnChange'] = df['open'] - df['close'].shift(-1)
    df['ovnChangePercent'] = (df['ovnChange'] / df['close'].shift(-1)) * 100
    df['futureCloseBin'] = df['futureClose'].apply(binary_up_down_)
    df.at[0, 'futureCloseBin'] = np.nan
    return df


class API:
    api_key = None

    def set_api_key(self, input_key: str):
        API.api_key = input_key
    def __init__(self):
        self.base_url = 'https://financialmodelingprep.com/api/v3/'
        self.api_key = API.api_key


class Search(API):
    def __init__(self, ticker: str):
        super().__init__()
        self.ticker = ticker.upper()

    def price(self, date_range=None, raw=False):
        if date_range is None:
            url = f'{self.base_url}historical-price-full/{self.ticker}?from=2000-01-01&to=2025-01-01&apikey={self.api_key}'
            r = requests.get(url)
            if r.status_code == 200:
                data = r.json()
                if not data:
                    return "ERROR! dictionary empty; check parameters"
                else:
                    if raw is False:
                        data = r.json()['historical']
                        df = pd.DataFrame(data)
                        df.drop(['adjClose', 'unadjustedVolume', 'vwap', 'label', 'changeOverTime'], axis=1, inplace=True)
                        df = data_shifts_(df)
                        return df
                    elif raw is True:
                        return data
                    else:
                        return "ERROR! raw parameter can only be boolian True of False"
            else:
                    return r.json()
            
        elif isinstance(date_range, list):
            check_dates = {}
            for date in date_range:
                if re.match(r"^(19\d{2}|20[0-2]\d)-(0[1-9]|1[0-2])-(0[1-9]|[12]\d|3[01])$", date):
                    check_dates[True] = date
                else:
                    check_dates[False] = date
            if False in check_dates:
                return "ERROR! date's are not properly formated in YYYY-MM-DD"
            else:
                url = f'{self.base_url}historical-price-full/{self.ticker}?from={date_range[0]}&to={date_range[1]}&apikey={self.api_key}'
                r = requests.get(url)
                if r.status_code == 200:
                    data = r.json()
                    if not data:
                        return "ERROR! dictionary empty; check parameters"
                    else:
                        if raw is False:
                            data = r.json()['historical']
                            df = pd.DataFrame(data)
                            df.drop(['adjClose', 'unadjustedVolume', 'vwap', 'label', 'changeOverTime'], axis=1, inplace=True)
                            df = data_shifts_(df)
                            return df
                        elif raw is True:
                            return data
                        else:
                            return "ERROR! raw parameter can only be boolian True of False"
                else:
                        return r.json()
                
        else:
            return "date_range must be a list of dates ['start', 'end'] and date must be in YYYY-MM-DD format"
        

class Lists(API):
    def __init__(self):
        super().__init__()
        url_screener = f'https://financialmodelingprep.com/api/v3/stock/list?apikey={self.api_key}'
        r_screener = requests.get(url_screener)
        df_screener = pd.DataFrame(r_screener.json())
        self.screener_all_ = df_screener

        url_industry = f'https://financialmodelingprep.com/api/v3/industries-list?apikey={self.api_key}'
        r_industry = requests.get(url_industry)
        df_industry = pd.DataFrame(r_industry.json())
        df_industry.rename(columns={0:'industry'}, inplace=True)
        self.industry_ = df_industry

        url_sector = f'https://financialmodelingprep.com/api/v3/sectors-list?apikey={self.api_key}'
        r_sector = requests.get(url_sector)
        df_sector = pd.DataFrame(r_sector.json())
        df_sector.rename(columns={0:'sector'}, inplace=True)
        self.sector_ = df_sector
    
    us_exchanges = [
    'NASDAQ',
    'OTC',
    'NYSE',
    'AMEX',
    'CBOE',
    ]

    def etfs(self):
        df = self.all_[self.all_['type'] == 'etf']
        df = df.reset_index(drop=True)
        return df
    
    def trusts(self):
        df = self.all_[self.all_['type'] == 'trust']
        df = df.reset_index(drop=True)
        return df

    def funds(self):
        df = self.all_[self.all_['type'] == 'fund']
        df = df.reset_index(drop=True)
        return df

    def stocks(self):
        df = self.all_[self.all_['type'] == 'stock']
        df = df.reset_index(drop=True)
        return df

    def us_etfs(self):
        df = self.all_[self.all_['type'] == 'etf']
        df = df[df['exchangeShortName'].isin(self.us_exchanges)]
        df = df.reset_index(drop=True)
        return df
    
    def us_trusts(self):
        df = self.all_[self.all_['type'] == 'trust']
        df = df[df['exchangeShortName'].isin(self.us_exchanges)]
        df = df.reset_index(drop=True)
        return df
    
    def us_funds(self):
        df = self.all_[self.all_['type'] == 'fund']
        df = df[df['exchangeShortName'].isin(self.us_exchanges)]
        df = df.reset_index(drop=True)
        return df
    
    def us_stocks(self):
        df = self.all_[self.all_['type'] == 'stock']
        df = df[df['exchangeShortName'].isin(self.us_exchanges)]
        df = df.reset_index(drop=True)
        return df
    
    def industries(self):
        url = f'https://financialmodelingprep.com/api/v3/industries-list?apikey={self.api_key}'
        r = requests.get(url)
        df = pd.DataFrame(r.json())
        df.rename(columns={0:'industry'}, inplace=True)
        return df
    
    def sectors(self):
        url = f'https://financialmodelingprep.com/api/v3/sectors-list?apikey={self.api_key}'
        r = requests.get(url)
        df = pd.DataFrame(r.json())
        df.rename(columns={0:'sector'}, inplace=True)
        return df

# New FMP Attempt

In [4]:
import requests                     # type: ignore
import pandas as pd                 # type: ignore

class FmpClient:
    def __init__(self, api_key):
        self.api_key = api_key
        self.base_url = 'https://financialmodelingprep.com/api/v3/'

    def request_(self, url):
        r = requests.get(url)
        if r.status_code == 200:
            data = r.json()
            if not data:
                return "ERROR! dictionary empty; check parameters"
            else:
                return data
        else:
            return r.json()

    def aggs_data_clean_(self, raw_data):
        data = raw_data['historical']
        df = pd.DataFrame(data)
        df.drop(columns=[
            'unadjustedVolume',
            'label',
            'changeOverTime',
            'adjClose'
        ], axis=1, inplace=True)
        df.rename(columns={
            'changePercent':'percent_change'
        }, inplace=True)
        df['date'] = pd.to_datetime(df['date'])
        df['change'] = round(df['change'], 4)
        df['percent_change'] = round(df['percent_change'], 4)
        df['vwap'] = round(df['vwap'], 4)
        return df

    def human_read_(self, number):
        suffixes = ['', 'K', 'M', 'B', 'T']
        magnitude = 0
        num = float(number)
        while abs(num) >= 1000 and magnitude < len(suffixes) - 1:
            magnitude += 1
            num /= 1000.0
        human_readable = f"{num:.3}{suffixes[magnitude]}"
        return human_readable

    ##############################
    """ 
    this section is for gathering historical and realtime aggregate data
    """    

    def get_aggs(self, ticker):
        url = f'{self.base_url}historical-price-full/{ticker.upper()}?from=2000-01-01&to=2025-01-01&apikey={self.api_key}'
        data = self.request_(url)
        df = self.aggs_data_clean_(data)
        df['ticker'] = ticker.upper()
        return df

    def get_price_rt(self, ticker):
        url = f'{self.base_url}quote-short/{ticker.upper()}?apikey={self.api_key}'
        data = self.request_(url)
        df = pd.DataFrame(data)
        return df

    def get_market_cap_rt(self, ticker, human=False):
        url = f'{self.base_url}market-capitalization/{ticker.upper()}?from=2000-01-01&to-2025-01-01&apikey={self.api_key}'
        data = self.request_(url)
        if data.empty:
            return
        else:
            df = pd.DataFrame(data)
            df.drop(columns=['date'], inplace=True)
            df.rename(columns={'marketCap':'market_cap'}, inplace=True)
            if human:
                df['market_cap'] = df['market_cap'].apply(self.human_read_)
                return df
            else:
                return df

    def get_market_cap_range(self, ticker, start, finish):
        url = f'{self.base_url}historical-market-capitalization/{ticker.upper()}?&from={start}&to={finish}&apikey={self.api_key}'
        data = self.request_(url)
        if data == "ERROR! dictionary empty; check parameters":
            return "ERROR! dictionary empty, no data available"
        else:
            df = pd.DataFrame(data)
            df.rename(columns={'marketCap':'market_cap'}, inplace=True)
            df['date'] = pd.to_datetime(df['date'])
            return df

    def get_market_cap_history(self, ticker):
        dates = [
            ['2000-01-01','2005-01-01'],
            ['2005-01-01', '2010-01-01'],
            ['2010-01-01', '2015-01-01'],
            ['2015-01-01', '2020-01-01'],
            ['2020-01-01', '2025-01-01']
        ]
        df = pd.DataFrame()
        for i in dates:
            data = self.get_market_cap_range(ticker.upper(), i[0], i[1])
            if isinstance(data, str) and data == "ERROR! dictionary empty, no data available":
                continue
            else:
                df = pd.concat([df, data], ignore_index=True)
        df = df.sort_values(by='date').reset_index(drop=True)
        return df
    
    ###########################

    def get_top_x_snp_companies(self, x):
        url = f'{self.base_url}stock-screener?isEtf=false&isFund=False&limit={x}&apikey={self.api_key}'
        r = requests.get(url)
        data = pd.DataFrame(r.json())
        return data

    ############################
    def sector_industry_data_clean_(self, data):
        df = pd.DataFrame(data)
        df = df[['symbol', 'companyName', 'exchangeShortName']]
        df.rename(columns={
            'companyName':'name',
            'exchangeShortName':'exchange'
        }, inplace=True)
        df = df[df['exchange'].isin(['NASDAQ', 'NYSE'])]
        df.reset_index(inplace=True, drop=True)
        df = df[['symbol', 'name']]
        return df

    def get_sectors(self):
        url = f'{self.base_url}sectors-list?apikey={self.api_key}'
        data = self.request_(url)
        data_sorted = sorted(data)
        return sorted(data)

    def get_industries(self):
        url = f'{self.base_url}industries-list?apikey={self.api_key}'
        data = self.request_(url)
        data_sorted = sorted(data)
        return data_sorted

    def sector_search(self, sector, simple=True):
        url = f'{self.base_url}stock-screener?apikey={self.api_key}&isEtf=false&isFund=false&isActivelyTrading=true&sector={sector}'
        data = self.request_(url)
        data_clean = self.sector_industry_data_clean_(data)
        return data_clean

    def industry_search(self, industry):
        url = f'{self.base_url}stock-screener?apikey={self.api_key}&isEtf=false&isFund=false&isActivelyTrading=true&industry={industry}'
        data = self.request_(url)
        data_clean = self.sector_industry_data_clean_(data)
        return data_clean
    
    def get_snp_500_companies(self):
        url = f'{self.base_url}sp500_constituent?apikey={self.api_key}'
        data = self.request_(url)
        df = pd.DataFrame(data)
        for index, row in df.iterrows():
            t = row['symbol']
            mkt_cap = self.get_market_cap_rt(t)
            row['mkt_cap'] = mkt_cap
        return df

    def get_dow_companies(self):
        pass



    #################################

    def income_statement_data_clean_(self, data):
        df = pd.DataFrame(data)
        df.rename(columns={
            'calendarYear':'year',
            'costOfRevenue':'cost_of_revenue',
            'grossProfit':'gross_profit',
            'grossProfitRatio':'gross_profit_ratio',
            'researchAndDevelopmentExpenses':'r_and_d_exp',
            'generalAndAdministrativeExpenses':'general_and_admin_exp',
            'sellingAndMarketingExpenses':'selling_and_marketing_exp',
            'sellingGeneralAndAdministrativeExpenses':'selling_general_and_admin_exp',
            'otherExpenses':'other_exp',
            'operatingExpenses':'operating_exp',
            'costAndExpenses':'cost_and_exp',
            'interestIncome':'interest_income',
            'interestExpense':'interest_expense',
            'depreciationAndAmortization': 'depreciation_and_amortization',
            'ebitdaratio':'ebitda_ratio',
            'operatingIncome':'operating_income',
            'operatingIncomeRatio':'operating_income_ratio',
            'totalOtherIncomeExpensesNet':'total_other_income_exp_net',
            'incomeBeforeTax':'income_before_tax',
            'incomeBeforeTaxRatio':'income_before_tax_ratio',
            'incomeTaxExpense':'income_tax_expense',
            'netIncome':'net_income',
            'netIncomeRatio':'net_income_ratio',
        }, inplace=True)

        df.drop(columns=[
            'reportedCurrency',
            'cik',
            'link',
            'date',
            'finalLink',
            'acceptedDate',
            'epsdiluted',
            'weightedAverageShsOut',
            'weightedAverageShsOutDil'
        ],inplace=True)
        df['date'] = pd.to_datetime(df['fillingDate'])
        df.drop(columns=['fillingDate'], inplace=True)
        df = df[df['date'].dt.year >= 2000]
        return df

    def get_income_statement_q(self, ticker):
        url = f'{self.base_url}income-statement/{ticker.upper()}?period=quarter&apikey={self.api_key}'
        data = self.request_(url)
        df = self.income_statement_data_clean_(data)
        return df

    def get_income_statement_a(self, ticker):
        url = f'{self.base_url}income-statement/{ticker.upper()}?period=quarter&apikey={self.api_key}'
        data = self.request_(url)
        return data
    
    #################################
    
    def income_statement_data_clean_(self, data):
        df = pd.DataFrame(data)
        df.rename(columns={
            'calendarYear':'year',
            'costOfRevenue':'cost_of_revenue',
            'grossProfit':'gross_profit',
            'grossProfitRatio':'gross_profit_ratio',
            'researchAndDevelopmentExpenses':'r_and_d_exp',
            'generalAndAdministrativeExpenses':'general_and_admin_exp',
            'sellingAndMarketingExpenses':'selling_and_marketing_exp',
            'sellingGeneralAndAdministrativeExpenses':'selling_general_and_admin_exp',
            'otherExpenses':'other_exp',
            'operatingExpenses':'operating_exp',
            'costAndExpenses':'cost_and_exp',
            'interestIncome':'interest_income',
            'interestExpense':'interest_expense',
            'depreciationAndAmortization': 'depreciation_and_amortization',
            'ebitdaratio':'ebitda_ratio',
            'operatingIncome':'operating_income',
            'operatingIncomeRatio':'operating_income_ratio',
            'totalOtherIncomeExpensesNet':'total_other_income_exp_net',
            'incomeBeforeTax':'income_before_tax',
            'incomeBeforeTaxRatio':'income_before_tax_ratio',
            'incomeTaxExpense':'income_tax_expense',
            'netIncome':'net_income',
            'netIncomeRatio':'net_income_ratio',
        }, inplace=True)

        df.drop(columns=[
            'reportedCurrency',
            'cik',
            'link',
            'date',
            'finalLink',
            'acceptedDate',
            'epsdiluted',
            'weightedAverageShsOut',
            'weightedAverageShsOutDil'
        ],inplace=True)
        df['date'] = pd.to_datetime(df['fillingDate'])
        df.drop(columns=['fillingDate'], inplace=True)
        df = df[df['date'].dt.year >= 2000]
        return df
    
    def get_balance_sheet(self):
        url = f'{self.base_url}balance-sheet-statement/AAPL?period=annual&apikey={self.api_key}'
        data = self.request_(url)
        return data

    def get_cash_flow(self):
        url = f'{self.base_url}cash-flow-statement/AAPL?period=annual&apikey={self.api_key}'
        data = self.request_(url)
        return data
    

